In [4]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
load_dotenv(override=True)

True

In [7]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [8]:
pushover_token= os.getenv("PUSHOVER_TOKEN")
pushover_user= os.getenv("PUSHOVER_USER")
pushover_url = f"https://api.pushover.net/1/messages.json"

def push (text: str):
    """ Send a push notification to the user """
    requests.post(pushover_url, data={"token": pushover_token, "user": pushover_user, "message": text})

tool_push = Tool(
    name="push",
    description="Send a push notification to the user",
    func=push
)

In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [11]:
for tool in tools:
    print (f'{tool.name} = {tool}')

click_element = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\samjo\AppData\Local\ms-playwright\chromium-1169\chrome-win\chrome.exe> version=136.0.7103.25>
navigate_browser = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\samjo\AppData\Local\ms-playwright\chromium-1169\chrome-win\chrome.exe> version=136.0.7103.25>
previous_webpage = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\samjo\AppData\Local\ms-playwright\chromium-1169\chrome-win\chrome.exe> version=136.0.7103.25>
extract_text = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\samjo\AppData\Local\ms-playwright\chromium-1169\chrome-win\chrome.exe> version=136.0.7103.25>
extract_hyperlinks = async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\samjo\AppData\Local\ms-playwright\chromium-1169\chrome-win\chrome.exe> version=136.0.7103.25>
get_elements = async_browser=<Browser ty

In [19]:
tool_dict = {tool.name:tool for tool in tools}
navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool = tool_dict.get("extract_text")
await navigate_tool.arun({"url":"https://www.cricinfo.com"})
text = await extract_text_tool.arun({})

In [ ]:

import textwrap
print (textwrap.fill(text, width=100))

In [21]:
all_tools = tools +[tool_push]

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)







